In [1]:
import glob
import pandas as pd
from pathlib import Path
import re

In [2]:
from PIL import Image

In [22]:
!ls /nas/chenyi/share/weights/detector_query/deploy/detector_query_dav4_1.pt

/nas/chenyi/share/weights/detector_query/deploy/detector_query_dav4_1.pt


In [3]:
root = Path('/nas/chenyi/datasets_cls/gallery_detect/')
det_root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/')

In [4]:
show_root = root /'gallery_oss/dadetv4/info/'
!ls $show_root

clred_final.csv       crops_imgs1m_list.json  dirty_data
clred_labels_oss.csv  crops_rev_list.json     @eaDir
clred_temp.csv	      dadetv4		      imgs_rev_list.json
clred_temp_final.csv  data_cls.ipynb	      rewrite_labels_plist.json


In [6]:
show_root = root /'gallery_oss/dadetv4/info/dadetv4/'
!ls $show_root

bboxes_all_df.csv    bboxes_detv4_temp_df.csv  imgs_det_df.csv	labels_df.csv
bboxes_detv4_df.csv  crops_all_df.csv	       imgs_org_df.csv


In [5]:
clred_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/clred_labels_oss.csv')

In [41]:
# bboxes_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_all_df.csv')

In [42]:
bboxes_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_detv4_df.csv')

In [43]:
len(bboxes_df)

6599176

In [44]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,933,933


### Dataselect

In [45]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,933,933


In [46]:
bboxes_df['fid'] = bboxes_df.apply(lambda x: '{0}_{1}'.format(Path(x['label_p']).stem, x['id_crop']), axis=1)

In [47]:
clred_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth,label_eng,crops_sp,crops_tp,fid
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,933,933,shoe,/nas/chenyi/datasets_cls/gallery_detect/galler...,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_0


In [48]:
clred_df['fid'] = clred_df.apply(lambda x: '{0}_{1}'.format(x['stem'], x['id_crop']), axis=1)

In [49]:
dirty_df = bboxes_df[~bboxes_df['fid'].isin(clred_df['fid'])]

In [50]:
len(clred_df), len(bboxes_df), len(dirty_df)

(4276565, 6599176, 2322611)

In [53]:
dirty_df.value_counts('label_pred')

label_pred
0    1116545
2     379363
1     299133
3     223523
5     220452
4      83595
dtype: int64

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth,fid
8,1,0.80761,0.589496,0.230439,0.233655,0.479052,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0
9,0,0.28671,0.987674,0.058950,0.024652,0.649600,1,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_1


#### 脏数据图片

In [54]:
len(dirty_df.drop_duplicates('stem'))

801151

In [ ]:
# dirty_df['crops_tp'] = 

#### 处理小图

In [19]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dirty_data/

dirty_crops_df.csv  select_candidate_df.csv


In [3]:
dirty_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dirty_data/dirty_crops_df.csv')
# dirty_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dirty_crops_df.csv', index=False)

In [4]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [5]:
model_label_map = {
    0: 'shoe',
    1: 'bag',
    2: 'upperbody',
    3: 'lowerbody',  # 裤子
    4: 'skirt',  # 裙子
    5: 'wholebody',  # 连体装
}

In [6]:
dirty_df.head(2)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth,fid
0,1,0.80761,0.589496,0.230439,0.233655,0.479052,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0
1,0,0.28671,0.987674,0.058950,0.024652,0.649600,1,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_1


In [7]:
dirty_df['label_eng'] = dirty_df['label_pred'].map(model_label_map)

In [8]:
dirty_df.value_counts('label_eng')

label_eng
shoe         1116545
upperbody     379363
bag           299133
lowerbody     223523
wholebody     220452
skirt          83595
dtype: int64

In [9]:
dirty_df.head(2)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth,fid,label_eng
0,1,0.80761,0.589496,0.230439,0.233655,0.479052,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0,bag
1,0,0.28671,0.987674,0.058950,0.024652,0.649600,1,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,450,450,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_1,shoe


##### 小图路径

In [9]:
# dirty_df['label_eng'] = dirty_df['label_pred'].map(model_label_map)

In [10]:
crops_root = '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops'
dirty_df['crops_sp'] = dirty_df.apply(lambda row:
                                                '{0}/{1}/{2}_{3}.jpg'.format(crops_root, row['label_eng'], row['stem'], row['id_crop']),
                                                axis=1)

In [11]:
crops_troot = '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops_dirty'
dirty_df['crops_tp'] = dirty_df.apply(lambda x:
                                                '{0}/{1}/{2}/{3}'.format(crops_troot, x['label_eng'], x['stem'][:2], Path(x['crops_sp']).name),
                                                axis=1)

In [12]:
dirty_df.head(2).values

array([[1, 0.80761, 0.589496, 0.230439, 0.233655, 0.479052, 0,
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/labels/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767.txt',
        '6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767',
        '/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767.jpg',
        450, 450, '6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0', 'bag',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/bag/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops_dirty/bag/69/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0.jpg'],
       [0, 0.28671, 0.987674, 0.0589496, 0.0246517, 0.6496, 1,
        '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/labels/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767.txt',
        '6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767',
        '/nas/os

### 数据拷贝

In [13]:
import sys
sys.path.append('../../')
from utils.data.transfer import copy_data_mkdir, run_task_multithread

In [14]:
input_list = [list(item) for item in zip(dirty_df['crops_sp'].values, dirty_df['crops_tp'].values)]

In [15]:
%%time
import shutil
def copy_data_mkdir(plist, mode='move'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if ps.is_file() and (not pt.is_file()):
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 3.58 µs


In [18]:
from tqdm import tqdm
for plist in tqdm(input_list):
    copy_data_mkdir(plist)
#     # break

100%|███████████████████████████████| 2322611/2322611 [18:09:30<00:00, 35.53it/s]


In [17]:
input_list[0]

['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/bag/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0.jpg',
 '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops_dirty/bag/69/6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767_0.jpg']

In [18]:
# %%time           
# run_task_multithread(input_list[::-1], copy_data_mkdir, 20)